# Part 1

--- Day 2: Password Philosophy ---

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

```
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
```

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, `1-3 a` means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, `cdefg`, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [15]:
from dataclasses import dataclass
from pathlib import Path
from typing import Generator, Tuple

In [11]:
INPUT_FILE: Path = Path.cwd() / 'inputs' / 'day02' / 'part1.txt'

In [9]:
def input_passwords() -> Generator[str, None, None]:
    with INPUT_FILE.open() as fp:
        for line in fp:
            yield line.strip()

In [5]:
@dataclass
class PasswordRule:
    letter: str
    min_: int
    max_: int

In [10]:
def parse_password_rule(rule: str) -> PasswordRule:
    min_max, letter = rule.split()
    min_, max_ = min_max.split('-')
    return PasswordRule(letter, int(min_), int(max_))
parse_password_rule('1-3 a')

PasswordRule(letter='a', min_=1, max_=3)

In [12]:
def count_letter(str_: str, letter: str) -> int:
    return sum(l == letter for l in str_)
count_letter('aacdfdaa', 'a')

4

In [14]:
def is_password_valid(password: str, rule: PasswordRule) -> bool:
    letter_count = count_letter(password, rule.letter)
    return rule.min_ <= letter_count <= rule.max_
is_password_valid('abcde', PasswordRule('a', 1, 3)), is_password_valid('cdefg', PasswordRule('b', 1, 3))

(True, False)

In [17]:
def parse_password_line(line: str) -> Tuple[str, PasswordRule]:
    rule, password = line.split(':')
    return password.strip(), parse_password_rule(rule)
parse_password_line('1-3 b: cdefg')

('cdefg', PasswordRule(letter='b', min_=1, max_=3))

In [19]:
sum(
    is_password_valid(password, rule)
    for password, rule in (parse_password_line(line) for line in input_passwords())
)

383

# Part 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

```
1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
```

How many passwords are valid according to the new interpretation of the policies?

In [21]:
def is_password_valid_p2(password: str, rule: PasswordRule) -> bool:
    return (
        (password[rule.min_ - 1] == rule.letter or password[rule.max_ - 1] == rule.letter)
        and (password[rule.min_ - 1] != password[rule.max_ - 1])
    )
(
    is_password_valid_p2('abcde', PasswordRule('a', 1, 3)),
    is_password_valid_p2('cdefg', PasswordRule('b', 1, 3)),
    is_password_valid_p2('ccccccccc', PasswordRule('c', 2, 9)),
)

(True, False, False)

In [22]:
sum(
    is_password_valid_p2(password, rule)
    for password, rule in (parse_password_line(line) for line in input_passwords())
)

272